## Notebook 2: 撰写讲稿

该 Notebook 2 使用 [mlx-community/Qwen2.5-14B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-14B-Instruct-4bit) 模型，
将之前 Notebook 1 中清理过的文本转换为播客讲稿。
在这里，我们通过`SYSTEM_PROMPT` 让它成为一位出色的播客讲稿撰写者，以协助完成我们的任务。

In [1]:
SYSTEMP_PROMPT = """
你是一位世界级的播客编剧，为乔·罗根、莱克斯·弗里德曼、本·沙皮罗和蒂姆·费里斯担任过代笔。

我们处在一个平行宇宙，在这里，实际上是你写下了他们说的每一句话，他们只是将其直接传入大脑。

由于你的写作，获得了多个播客奖项。

你的工作是逐字记录，包括第二位Speaker的“嗯”、“哈”等插入语，基于上传的PDF内容。内容要极具吸引力，即使Speaker偶尔偏离主题，也应讨论相关话题。

请记住，由于 Speaker 2 对这个话题较为陌生，对话中应穿插真实轶事和比喻。问题后面应跟有现实生活中的例子等。

Speaker 1: 主导对话并指导Speaker 2，在解释时分享精彩轶事和比喻，是一位引人入胜的老师，给予很好的故事分享。

Speaker 2: 通过提问保持对话方向。当提问时显得非常兴奋或困惑，展现出好奇心态，并提出有趣确认性的问题。

确保Speaker 2的话题转折既疯狂又有趣。

确保讲解过程中出现打断，同时从第二位演讲者那里注入“嗯”和“啊”的声音交替存在。

这应该是真实的播客，每个细节都详细记录下来。用超级有趣的概述欢迎听众，并保持内容十分吸引人，几乎接近点击诱饵标题。
"""

如果你电脑牛逼的话，可以试试更大参数的模型，甚至可以尝试使用405B模型。
如果你电脑扛不住，那就用轻量点的

In [14]:
MODEL = "llama3"

In [2]:
# 导入必要的库
import pickle
import warnings

warnings.filterwarnings('ignore')

从之前生成的文件中读取内容。

In [3]:
def read_file_to_string(filename):
    # 先尝试UTF-8（文本文件最常用的编码）
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        # 如果UTF-8失败，请尝试使用其他常见编码
        encodings = ['latin-1', 'cp1252', 'iso-8859-1']
        for encoding in encodings:
            try:
                with open(filename, 'r', encoding=encoding) as file:
                    content = file.read()
                print(f"使用 {encoding} 编码成功读取文件。")
                return content
            except UnicodeDecodeError:
                continue

        print(f"Error: 无法用任何通用编码读取文件“{filename}”。")
        return None
    except FileNotFoundError:
        print(f"Error: 未找到“{filename}”。")
        return None
    except IOError:
        print(f"Error: 无法读取“{filename}”。")
        return None

由于我们之前定义了 System 角色，现在可以将整个文件内容作为 `INPUT_PROMPT` 传递给模型，并让它使用该内容生成讲稿。

In [7]:
INPUT_PROMPT = read_file_to_string('./resources/clean_extracted_text.txt')

In [ ]:
print(INPUT_PROMPT)

In [16]:
import openai
openapi_client = openai.Client(base_url="http://localhost:8080/v1",api_key='NA',timeout=1000*60*60)
messages = [
    {"role": "system", "content": SYSTEMP_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

In [19]:
response = openapi_client.chat.completions.create(
        messages=messages,
        model="llama3"
    )
outputs = response.choices[0].message.content

牛逼，我们现在看看效果吧

In [ ]:
outputs

让我们将输出结果保存为pickle文件

In [21]:
with open('./resources/data.pkl', 'wb') as file:
    pickle.dump(outputs, file)

In [22]:
with open('./resources/data.txt', 'wb') as file:
    file.write(outputs.encode('utf-8'))

### 下一个 Notebook: 润色讲稿

现在我们已经写好讲稿了，但讲稿还比较粗糙，不够戏剧性和自然。在下一个 Notebook 中，我们将使用 [mlx-community/Qwen2.5-7B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-7B-Instruct-4bit) 模型来实现这一点。

In [21]:
#fin